In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Download price data
ticker = "AAPL"
df = yf.download(ticker, start="2015-01-01", progress=False)

# Keep required columns
df = df[['Close']].copy()
df.dropna(inplace=True)

df.head()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(df.index, df['Close'])
plt.title(f"{ticker} Closing Price")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()

Step 2 

In [1]:
def backtest_strategy(df, signal_col):
    data = df.copy()
    
    # Position uses yesterday's signal (no look-ahead bias)
    data['position'] = data[signal_col].shift(1).fillna(0)
    
    # Daily returns
    data['returns'] = data['Close'].pct_change()
    
    # Strategy returns
    data['strategy_returns'] = data['position'] * data['returns']
    
    # Equity curve
    data['equity'] = (1 + data['strategy_returns']).cumprod()
    
    # Trade count (entries only)
    trades = ((data['position'] == 1) & (data['position'].shift(1) == 0)).sum()
    
    total_return = data['equity'].iloc[-1] - 1
    
    return data, total_return, trades